# Path and Imports

In [1]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset
import wavio
import numpy as np
from IPython.display import Audio
import nltk
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import IPython.display as ipd
import librosa

## Play Audio

In [31]:
# pip install playsound
# audio='/home/shashank/.cache/huggingface/datasets/downloads/extracted/7fa26e7577c7aa8fcc6b61ad54d91601f4287951b90bca42b8f58b8d9b7fd2ab/dev_clean/1272/128104/1272-128104-0000.flac'
audio='/media/shashank/data1/iisc/Capstone/results/p228xp227.wav'
audio='/media/shashank/data1/iisc/Capstone/wavs/s/p228/p228_003.wav'
audio='/media/shashank/data1/iisc/Capstone/style_conversions/236xp232.wav'

audio = '/media/shashank/data1/iisc/projects/capstone_project/templates/assets/audio_files/VCTK-corpus/p240/p240_023.wav'
audio = '/media/shashank/data1/iisc/projects/StarGANv2-VC-main/Demo/results/style_conversions/225xp227.wav'
audio = '/media/shashank/data1/iisc/projects/capstone_project/templates/assets/audio_files/knn_results/p228xp225.wav'


In [24]:
/media/shashank/data1/iisc/projects/capstone_project/templates/assets/audio_files/style_conversions

'/media/shashank/data1/iisc/project/capstone_project/static_files/audio_files/p226/p226_003.wav'

In [32]:
# Audio(audio)
sr=16000
# Audio(audio, rate=16000)
display(ipd.Audio(audio, rate=sr))

# Speech to Text Conversion

In [4]:
speech_to_text_model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
wave, sample_rate = librosa.load(audio, sr=sr)
audio, index = librosa.effects.trim(wave, top_db=30)
if sample_rate != sr:
    wave = librosa.resample(wave, sample_rate, sr)

In [52]:
# audio
# w = wavio.read(audio)
# print(w.rate)
# audio.data.shape
# # audio.rate

array([-0.00970742, -0.00952158, -0.00962916, ...,  0.00071846,
        0.00115868,  0.00121708], dtype=float32)

In [34]:
inputs = processor(audio, sampling_rate=sr, return_tensors="pt")
generated_ids = speech_to_text_model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
print(transcription)

['sixpence of fresh snow peas five thick slabs of blue cheese and maybe a snack for her brother bob']


In [13]:
source_text='six spoons of fresh beans five thick slabs of goat cheese and maybe a snack for her brother Bob'
# source_text='please take the shape of a long round arch with its path high above and its two ends apparently be on the horizon'

In [14]:
model_name = "bert-base-cased-finetuned-mrpc"
tokenizer = AutoTokenizer.from_pretrained(model_name)
similarity_model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokens = tokenizer.encode_plus(source_text, transcription[0], return_tensors="pt")
classification_logits = similarity_model(**tokens)[0]
results = torch.softmax(classification_logits, dim=1).tolist()[0]

classes = ["Not Similar", "Similar"]
print(source_text, '\n', transcription[0])
for i in range(len(classes)):
    print(f"{classes[i]}: {round(results[i] * 100)}%")


six spoons of fresh beans five thick slabs of goat cheese and maybe a snack for her brother Bob 
 six springs of fresh mybies five thick snaps of great cheese and lady herself are valuable
Not Similar: 72%
Similar: 28%


# BLEU Score of Translation 

In [19]:
smoothie = SmoothingFunction().method4
print('BLEU Score:', round(bleu(transcription, source_text, smoothing_function=smoothie)* 100, 2), '%')

BLEU Score: 56.23 %


In [89]:
source_text = 'if the red of the second bow falls upon the green of the first the result is to give a bow with an abnormally wide yellow band since red and green light when mixed form yellow'


In [87]:
transcription = ['its the right of the second bow falls upon the green of the first the result is together bow with an abnormally wide yellow burnt since red and green knight or mixed four narrow', 'if the red of the second bow falls upon the green of the first the result is to give a bow with an abnormally wide yellow band since red and green light on mixed form yellow','if the matter the second blow falls upon the green of the first the result is to give a bow with an abnormally wide yellow bound since red and green lights were next form yellow']


In [22]:
smoothie = SmoothingFunction().method4

for item in transcription:
    print('BLEU Score:', round(bleu(item, source_text, smoothing_function=smoothie)* 100, 2), '%')

BLEU Score: 0.75 %
BLEU Score: 0.75 %
BLEU Score: 0.75 %


In [15]:
source_text = "six spoons of fresh snow peas five thick slabs of blue cheese and maybe a snack for her brother bob"


In [35]:
transcription = ["six spoons of fresh snow peas five thick slabs of blue cheese and maybe a stack for her brother bob", 
                 "six spoons of fresh snipes five thick slabs of blue cheese and maybe a stack for her brother bob", 
                 "sixpence of fresh snow peas five thick slabs of blue cheese and maybe a snack for her brother bob"]

In [36]:
for item in transcription:
    print('BLEU Score:', round(bleu(item, source_text, smoothing_function=smoothie)* 100, 2), '%')

BLEU Score: 0.75 %
BLEU Score: 0.74 %
BLEU Score: 0.75 %
